# City Raininess

##### This notebook allows a user to enter a city and country, and will locate the latitude and longitude of the most populated city following those parameters,if there is one; this utilizes the world_cities CSV. Then, based on the latitude and longitude of the city, historical and forecast data will be determined from the OpenMeteo API. A raininess index is defined, weighting recent raininess more heavily than less recent data.

***

In [1]:
# Importing
import pandas as pd
import json
import requests
import datetime

***

### 1. Converting City and Country to Latitude and Longitude

In [3]:
def get_city_latlon(city, country):
    '''
    Get the latitude and longitude of a city in a country, derived from the world_cities CSV file.
    Params:
        city: str, the name of the city
        country: str, the name of the country
    Returns:
        a tuple of the latitude and longitude
    '''
    # Read the world_cities CSV file
    world_cities = pd.read_csv('world_cities.csv')

    # Filter the dataframe based on the city and country
    filtered_cities = world_cities[(world_cities['name'] == city) & (world_cities['country'] == country)]

    # Check if any matching cities are found
    if len(filtered_cities) > 0:
        # Get the longitude of the first matching city
        longitude = filtered_cities.iloc[0]['lng']
        # Get the latitude of the first matching city
        latitude = filtered_cities.iloc[0]['lat']
        # Return the latitude and longitude as a tuple
        return (latitude, longitude)
    else:
        # Return None if no matching city is found
        return None

### <center>Testing</center>

In [4]:
# Testing the above function

# Test case 1: Mumbai, India
city = 'Mumbai'
country = 'IN'
print(f'The latitude and longitude of {city}, {country} is {get_city_latlon(city, country)}')

# Test case 2: New York City, United States
city = 'New York City'
country = 'US'
print(f'The latitude and longitude of {city}, {country} is {get_city_latlon(city, country)}')

# Test case 3: Tokyo, Japan
city = 'Tokyo'
country = 'JP'
print(f'The latitude and longitude of {city}, {country} is {get_city_latlon(city, country)}')

# Test case 4: Walnut Creek, United States
city = 'Walnut Creek'
country = 'US'
print(f'The latitude and longitude of {city}, {country} is {get_city_latlon(city, country)}')

The latitude and longitude of Mumbai, IN is (19.07283, 72.88261)
The latitude and longitude of New York City, US is (40.71427, -74.00597)
The latitude and longitude of Tokyo, JP is (35.6895, 139.69171)
The latitude and longitude of Walnut Creek, US is (37.90631, -122.06496)


***

### 2. Collecting Historical Data (Last Year)

In [20]:
def get_historical_precipitation(latitude: float, longitude: float) -> dict:
    '''
    Get the number of days rained and number of mm of rain in the past year
    Params:
        latitude: float - latitude of the location
        longitude: float - longitude of the location
    Returns:
        a dictionary of the daily rain for the past five years in mm, number of hours of precipitation, and the mean probability of precipitation
    '''
    
    # Get the historical weather data

    base_historical_url = "https://archive-api.open-meteo.com/v1/era5?"
    params_lat_long_ = "latitude=" + str(latitude) + "&longitude=" + str(longitude)

    # Get the current date
    current_date = datetime.date.today()
    
    # Calculate the date five years ago
    five_years_ago = current_date - datetime.timedelta(days=365*5)
    
    # Format the date as YYYY-MM-DD
    formatted_date = five_years_ago.strftime("%Y-%m-%d")
    
    params_dates = "&start_date=" + formatted_date + "&end_date=" + str(current_date)
    param_rain_sum = "&daily=rain_sum&daily=precipitation_hours&daily=precipitation_probability_mean"
    
    total_url = base_historical_url + params_lat_long_ + params_dates + param_rain_sum

    response = requests.get(total_url)

    historical_data = response.json()
    
    historical_rain_sum = historical_data['daily']['rain_sum']
    historical_precipitation_hours = historical_data['daily']['precipitation_hours']
    historical_precipitation_probability = historical_data['daily']['precipitation_probability_mean']
    
    return {historical_rain_sum, historical_precipitation_hours, historical_precipitation_probability}

### <center>Testing</center>

In [21]:
# Testing historical precipitation using the get_historical_precipitation function and the get_city_latlon function

# Test 1: San Francisco
city = 'San Francisco'
country = 'US'
latitude, longitude = get_city_latlon(city, country)
print(get_historical_precipitation(latitude, longitude))

# Test 2: New York
city = 'New York City'
country = 'US'
latitude, longitude = get_city_latlon(city, country)
print(get_historical_precipitation(latitude, longitude))

# Test 3: London
city = 'London'
country = 'GB'
latitude, longitude = get_city_latlon(city, country)
print(get_historical_precipitation(latitude, longitude))

TypeError: unhashable type: 'list'

***

### 3. Collecting Forecast Data (Next 7 Days)

In [15]:
def get_forecast_precipitation(latitude: float, longitude: float) -> dict:
    '''
    Get the hourly forecast precipitation for a given latitude and longitude for the next 7 days
    Params: 
        latitude: float - latitude of the location
        longitude: float - Longitude of the location
    Returns:
        dict - Forecast precipitation
    '''

    # Building the base URL
    base_forecast_url = "https://api.open-meteo.com/v1/forecast?"
    params_lat_long = "latitude=" + str(latitude) + "&longitude="  + str(longitude)
    params_others = "&daily=rain_sum"

    final_url = base_forecast_url + params_lat_long + params_others

    # Getting the forecast data
    response = requests.get(final_url)

    # Extracting the forecast precipitation
    forecast_data = response.json()
    forecast_precipitation = forecast_data['daily']['rain_sum']
    return forecast_precipitation

### <center>Testing</center>

In [17]:
# Testing the get_forecast_precipitation function using the get_city_latlon function

# Test 1: San Francisco
city = 'San Francisco'
country = 'US'
latitude, longitude = get_city_latlon(city, country)
print(get_forecast_precipitation(latitude, longitude))

# Test 2: New York
city = 'New York City'
country = 'US'
latitude, longitude = get_city_latlon(city, country)
print(get_forecast_precipitation(latitude, longitude))

# Test 3: London
city = 'London'
country = 'GB'
latitude, longitude = get_city_latlon(city, country)
print(get_forecast_precipitation(latitude, longitude))

# Test 4: Mumbai
city = 'Mumbai'
country = 'IN'
latitude, longitude = get_city_latlon(city, country)
print(get_forecast_precipitation(latitude, longitude))

# Test 5: Tokyo
city = 'Tokyo'
country = 'JP'
latitude, longitude = get_city_latlon(city, country)
print(get_forecast_precipitation(latitude, longitude))

[0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[22.7, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0]


***

### 4. Defining the Raininess Index

##### The raininess index combines the daily historical data of amount of rain (mm), hours of precipitation, and the mean probability of precipitation with the hourly forecast data of amount of precipitation